In [12]:
import altair as alt
import altair_viewer
import json
import pandas as pd

In [13]:
with open('UA_FULL_Ukraine.geojson') as f:
    data = json.loads(f.read())

geo = pd.DataFrame(data['features'])

In [14]:
geo.head()

,type,geometry,properties
0,Feature,"{'type': 'Polygon', 'coordinates': [[[32.06948...","{'@id': 'relation/91278', 'admin_level': '4', ..."
1,Feature,"{'type': 'Polygon', 'coordinates': [[[33.29431...","{'admin_level': '4', 'boundary': 'administrati..."
2,Feature,"{'type': 'Polygon', 'coordinates': [[[25.74232...","{'admin_level': '4', 'boundary': 'administrati..."
3,Feature,"{'type': 'Polygon', 'coordinates': [[[34.97755...","{'admin_level': '4', 'boundary': 'administrati..."
4,Feature,"{'type': 'Polygon', 'coordinates': [[[34.98391...","{'admin_level': '4', 'boundary': 'administrati..."


In [15]:
# Ukraine regions background
background = alt.Chart(geo).mark_geoshape(fill='lightgray', stroke='white')

# event positions on background
ukr_df = pd.read_csv("2022-02-01-2023-02-01-Europe-Ukraine.csv", low_memory = False)
ukr_df = ukr_df.groupby(['admin2', 'latitude', 'longitude']).size().reset_index(name='lat_count')

points = alt.Chart(ukr_df).transform_aggregate(
    latitude='mean(latitude)',
    longitude='mean(longitude)',
    count='sum(lat_count)',
    groupby=['admin2']
).mark_circle().encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    size=alt.Size('count:Q', title='Number of events'),
    #color=alt.value('steelblue'),
    color=alt.Color('count:Q', scale=alt.Scale(scheme='inferno')),
    tooltip=[alt.Tooltip('admin2:N', title='Region'), 
             alt.Tooltip('count:Q', title='Events')]
).properties(
    #title='Events by region',
    width=800,
    height=500
)

altair_viewer.display(background + points)

Displaying chart at http://localhost:50700/

In [16]:
ukr_df = pd.read_csv("2022-02-01-2023-02-01-Europe-Ukraine.csv", low_memory = False)
ukr_df = ukr_df[(ukr_df['event_type'] == 'Battles') | (ukr_df['event_type'] =='Explosions/Remote violence')]
ukr_df['date'] = pd.to_datetime(ukr_df['event_date'])
ukr_df = ukr_df.groupby(['date', 'event_type']).size().reset_index(name='event_count')

event_chart = alt.Chart(ukr_df).mark_line().encode(
    alt.X('date:T', title='Date'),
    alt.Y('event_count:Q', title='Event count'),
    color=alt.Color('event_type:N', title='Event type'),
    ).properties(
    #title='Events by region',
    width=800,
    height=400
)
#event_chart
altair_viewer.display(event_chart)

Displaying chart at http://localhost:50700/